In [164]:
import pandas as pd
import urllib.request, json
import pydotplus as pydotplus
from sklearn import preprocessing, metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 
from IPython.display import Image 
import numpy as np

Funktsioon, millega saada andmed lehelt kätte. Tagastab andmed kujul [{}, {}, ... ,{}] ehk sõnastikke sisaldav massiiv, kus iga sõnastik hoiab infot ühe taotluse kohta


In [165]:
def getAndmed():
    with urllib.request.urlopen("https://toetused.kul.ee/public/applications/fetch?sort=submission_date%7Cdesc&page=1&per_page=20000000&applicant=&organization=&applicationround=&submissionDate=%7B%22start%22:null,%22end%22:null%7D") as f:
        data = json.load(f)
    andmed = data.get("data")
    return andmed

Tabeli tegemine

In [296]:
def tabel_andmetest():
    rawAndmed = getAndmed()
    
    id = []
    application_code = []
    applicationround_title = []
    approved_amount =[]
    cost_statement_submission_date = []
    cost_statement_submission_deadline = []
    managing_organization_name = []
    name = []
    project_name = []
    registration_date = []
    registry_code = []
    requested_amount = []
    status = []
    status_txt = []
    submission_date = []
    domain_code = []
    domain_name =[]
    
    for taotlus in rawAndmed:
        if taotlus.get("status") == "decision_ok":
            taotlus["status"] = 1
        elif taotlus.get("status") == "decision_not_ok":
            taotlus["status"] = 0
        else:
            taotlus["status"] = None
        
    
    for taotlus in rawAndmed:
        id.append(taotlus.get("id"))
        application_code.append(taotlus.get("application_code"))
        applicationround_title.append(taotlus.get("applicationround_title"))
        approved_amount.append(taotlus.get("approved_amount"))
        cost_statement_submission_date.append(taotlus.get("cost_statement_submission_date"))
        cost_statement_submission_deadline.append(taotlus.get("cost_statement_submission_deadline"))
        managing_organization_name.append(taotlus.get("managing_organization_name"))
        name.append(taotlus.get("name"))
        project_name.append(taotlus.get("project_name"))
        registration_date.append(taotlus.get("registration_date"))
        registry_code.append(taotlus.get("registry_code"))
        requested_amount.append(taotlus.get("requested_amount"))
        status.append(taotlus.get("status"))
        status_txt.append(taotlus.get("status_txt"))
        submission_date.append(taotlus.get("submission_date"))
        domain_code.append(taotlus.get("domain_code"))
        domain_name.append(taotlus.get("domain_name"))
    
    andmed = pd.DataFrame(
        {#"ID" : id,
        "application_code" : application_code,
        "applicationround_title" : applicationround_title,
        "approved_amount" : approved_amount,
        "cost_statement_submission_date" : cost_statement_submission_date,
        "cost_statement_submission_deadline" : cost_statement_submission_deadline,
        "managing_organization_name" : managing_organization_name,
        "name" : name,
        "project_name" : project_name,
        "registration_date" : registration_date,
        "registry_code" : registry_code,
        "requested_amount" : requested_amount,
        "status" : status,
        "status_txt" : status_txt,
        "submission_date" : submission_date,
        "domain_code" : domain_code,
        "domain_name" : domain_name
        }
    )
    return andmed

def decisionTree():
    tabel = tabel_andmetest()
    
    feature_cols = ["domain_name", "requested_amount"]
    treenimiseks = tabel[(tabel.status.notnull())]
    
    X = treenimiseks[feature_cols]
    y = treenimiseks["status"]
    
    enc1 = preprocessing.LabelEncoder()
    enc2 = preprocessing.LabelEncoder()
    domain_name_encoder = enc1.fit(tabel.domain_name.unique())
    managing_org_name_encoder = enc2.fit(tabel.managing_organization_name.unique())
    domain_int = domain_name_encoder.transform(X.domain_name)
    #managing_organization_name_int = managing_org_name_encoder.transform(X.managing_organization_name)
    X.domain_name = domain_int
    #X.managing_organization_name = managing_organization_name_int
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    dtc = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0, presort=False, random_state=None,
            splitter='best')
    
    dtc = dtc.fit(X_train, y_train)
    
    y_pred = dtc.predict(X_test)
    print(y_test, y_pred)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    import os
    
    os.environ['PATH'] = os.environ['PATH']+';'+os.environ['CONDA_PREFIX']+r"\Library\bin\graphviz"
    dot_data = StringIO()
    export_graphviz(dtc, out_file=dot_data,  
                    filled=True, rounded=True,
                    special_characters=True,feature_names = feature_cols,class_names=['0','1'])
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    graph.write_png('pilt.png')
    Image(graph.create_png())
    
    return dtc

def arva(arvamiseks):
    feature_cols = ["domain_name", "requested_amount"]
    X = arvamiseks[feature_cols]
    
    enc1 = preprocessing.LabelEncoder()
    domain_name_encoder = enc1.fit(tabel.domain_name.unique())
    domain_int = domain_name_encoder.transform(X.domain_name)
    X.domain_name = domain_int
    
    
    õpetaja = decisionTree()
    tulemus = õpetaja.predict(X)
    return tulemus

tabel = tabel_andmetest()
arvamiseks = tabel[(tabel.status.isnull())]

decisionTree()

C:\Users\enrih\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


1600    0.0
949     1.0
1895    1.0
1839    1.0
697     0.0
1280    1.0
1817    1.0
1331    1.0
1190    0.0
780     1.0
1674    0.0
1559    1.0
909     0.0
532     0.0
889     0.0
1923    1.0
590     0.0
1337    1.0
1462    1.0
1834    1.0
1831    1.0
1248    0.0
1878    0.0
1516    1.0
1825    1.0
974     1.0
1385    1.0
882     0.0
1683    1.0
1824    1.0
       ... 
972     1.0
1778    0.0
1197    1.0
803     0.0
1124    1.0
987     0.0
1050    1.0
742     1.0
712     1.0
536     0.0
1355    1.0
1854    1.0
1904    1.0
1552    1.0
1278    1.0
1527    0.0
1975    1.0
1068    0.0
1788    0.0
875     0.0
1579    1.0
1245    0.0
1325    1.0
919     0.0
866     0.0
964     0.0
1859    0.0
1084    0.0
858     1.0
1419    1.0
Name: status, Length: 285, dtype: float64 [0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0.
 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1.
 0. 1. 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0, presort=False, random_state=None,
            splitter='best')

In [167]:
def biggestFounding():
    data = tabel_andmetest();
    most_expensive = data.loc[data['approved_amount'].idxmax()]
    applicationround_title = most_expensive["applicationround_title"]
    approved_amount = most_expensive["approved_amount"]
    managing_organization_name = most_expensive["managing_organization_name"]
    name = most_expensive["name"]
    project_name = most_expensive["project_name"]
    return applicationround_title, approved_amount, managing_organization_name, name, project_name
    


In [168]:
def smallestFounding():
    data = tabel_andmetest();
    cheapest = data.loc[data['approved_amount'].idxmin()]
    applicationround_title = cheapest["applicationround_title"]
    approved_amount = cheapest["approved_amount"]
    managing_organization_name = cheapest["managing_organization_name"]
    name = cheapest["name"]
    project_name = cheapest["project_name"]
    return applicationround_title, approved_amount, managing_organization_name, name, project_name


('Ida-Virumaa ettevõtlust edendavad spordi- ja kultuuriüritused', 0.0, 'Integratsiooni Sihtasutus', 'Mittetulundusühing Õpilasesinduste Nõukogu', 'Jõuluküla')


In [ ]:
#Väike kasutajaliides 


